In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import pandas as pd
import numpy as np

import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from metric_learn import NCA
from sklearn.model_selection import cross_val_score, train_test_split

from sklearn.svm import SVC

from gensim.models import FastText
from gensim.test.utils import common_texts
from tpot import TPOTClassifier

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append(f'../../')

from src.data.sentence_vectorizer import SentenceVectorizer

Create random configuration
create config: 'temp/run_config_default_run_name.yml'


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kaigorodov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kaigorodov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
labels = pd.read_csv('../../data/interim/meddra_codes_terms_synonims.csv')
labels = labels['CODE']
meddra_labels = {v:k for k, v in enumerate(labels.unique())}

results = {
    'vectorizer': [],
    'train_model': [],
    'smm4h21': [],
    'smm4h17': [],
    'psytar': [],
    'cadec': [],
}

SIZE = 100
sv = SentenceVectorizer()
for vectorizer_name in sv.get_availables_vectorizers():
    print(f"vectorizer: {vectorizer_name}")
    results['vectorizer'] += [vectorizer_name] * 4
    path = '../../data/interim/'
    for name_train in os.listdir(path):

        if name_train not in ['smm4h17', 'smm4h21', 'psytar', 'cadec']:
            continue
        print(f"work with {name_train}")
        results['train_model'].append(name_train)

        folder = os.path.join(path, name_train)
        corpus_train = folder + '/train.csv'
        train = pd.read_csv(corpus_train)
        
        train = sv.vectorize(train, vectorizer_name=vectorizer_name) 

        X_train, y_train = train['term_vec'], train['code']
        X_train = pd.DataFrame([pd.Series(x) for x in X_train])
        y_train = y_train.apply(lambda x: int(meddra_labels[x]))

        #clf = make_pipeline(NCA(), SVC(gamma='scale'))
        clf = SVC(kernel='poly', gamma='scale')
        clf.fit(X_train, y_train)

        for name_test in os.listdir(path):
            if name_test not in ['smm4h17', 'smm4h21', 'psytar', 'cadec']:
                continue
            folder = os.path.join(path, name_test)
            corpus_train = folder + '/train.csv'
            corpus_test = folder + '/test.csv'
            train, test = pd.read_csv(corpus_train)[:1], pd.read_csv(corpus_test)
#            sv = SentenceVectorizer()
            _, test = sv.vectorize(train, test, vectorizer_name=vectorizer_name) 
            X_test, y_test = test['term_vec'], test['code']
            X_test = pd.DataFrame([pd.Series(x) for x in X_test])
            y_test = y_test.apply(lambda x: int(meddra_labels[x]))

            score = clf.score(X_test, y_test)
            print(f'\ttest with {name_test} score:', score)
            results[name_test].append(score)
        print()
    
results = pd.DataFrame(results)
results

vectorizer: fasttext_default_100
work with smm4h21


  0%|          | 1/1712 [00:00<00:24, 69.68it/s]


AxisError: axis 0 is out of bounds for array of dimension 0

In [ ]:
results

In [ ]:
# generations = 2
# population_size = 50
# max_eval_time_mins = 2
# n_jobs = 10
# max_iter = 10

# tpot = TPOTClassifier(generations=generations, 
#                       population_size=population_size,
#                       verbosity=2, 
#                       random_state=42, 
#                       max_eval_time_mins=max_eval_time_mins, 
#                       n_jobs=n_jobs)

# tpot.fit(X_train, y_train)
# score = tpot.score(X_test, y_test)
# score